# Travel Assistant Demo - OpenSearch Agentic Memory with Strands Agent

## Introduction

This notebook demonstrates how to build an intelligent travel assistant using OpenSearch's agentic memory capabilities integrated with Strands agent. The travel assistant showcases **long-term memory** functionality, enabling it to remember user preferences, past conversations, and travel history across multiple sessions.

### Key Features

- **Persistent Memory**: Remembers user preferences and travel history across sessions
- **Semantic Search**: Retrieves relevant memories based on context and meaning
- **Travel Intelligence**: Provides personalized recommendations based on past interactions
- **Session Continuity**: Maintains context even when conversations are resumed later

### What You'll Learn

- How to set up OpenSearch agentic memory for long-term storage
- Integration patterns between Strands agents and OpenSearch
- Building conversational AI that learns and remembers user preferences
- Implementing semantic memory retrieval for personalized experiences

## Prerequisites

To use this notebook, you'll need:
- Python 3.10+
- OpenSearch 3.3.2+
- AWS Credentials for Amazon Bedrock access

## Step 1: Install Dependencies and Setup

Install dependencies:

In [1]:
!pip install -qr ../requirements.txt

Set OpenSearch domain configuration:

In [2]:
opensearch_url = 'https://localhost:9200'
opensearch_username = 'admin'
opensearch_password = 'your_opensearch_password'
opensearch_verify_ssl = False

Set model configuration:

In [9]:
embedding_model_id = ''  # Leave empty to auto-create Amazon Titan Embedding model
llm_model_id = ''        # Leave empty to auto-create Amazon Bedrock Claude model

Set memory and session configuration:

In [6]:
# Modify as needed
memory_container_name = 'travel_memory'
session_id = 'travel_session' 
user_id = 'travel_user'
agent_id = 'travel_agent'

Set AWS credentials (Required for Amazon Bedrock access):

In [7]:
import os

os.environ['AWS_REGION'] = os.getenv('AWS_REGION', 'your_aws_region')
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID', 'your_access_key')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY', 'your_secret_key')
os.environ['AWS_SESSION_TOKEN'] = os.getenv('AWS_SESSION_TOKEN', '')  # Optional

## Step 2: Initialize Memory Provider

In [ ]:
import urllib3
import sys

sys.path.append('..')
from commons.opensearch_memory_tool import OpenSearchMemoryToolProvider

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


memory_provider = OpenSearchMemoryToolProvider(
    cluster_url=opensearch_url,
    username=opensearch_username,
    password=opensearch_password,
    verify_ssl=opensearch_verify_ssl,
    memory_container_name=memory_container_name,
    session_id=session_id,
    agent_id=agent_id,
    user_id=user_id,
    embedding_model_id=embedding_model_id,
    llm_id=llm_model_id
)

## Step 3: Create Travel Agent Assistant with Long-term Memory

In [50]:
from strands import Agent
from strands.models import BedrockModel

def start_travel_session(session_name):
    """Start a travel agent conversation session"""
    # Update session for memory provider
    memory_provider.session_id = session_name
    
    # Configure the model and agent
    model = BedrockModel(model_id="global.anthropic.claude-opus-4-5-20251101-v1:0")
    agent = Agent(
        model=model,
        tools=memory_provider.tools,
        system_prompt="""You are a professional travel agent with access to client memory. 
        Always store important client information (preferences, budget, restrictions) using the opensearch_memory tool.
        When clients return, search your memory to provide personalized recommendations."""
    )
    
    print(f"🌍 Travel Agent Ready - Session: {session_name}")
    print("Type 'q' or 'quit' to end the conversation\n")
    
    while True:
        question = input("👤 You: ").strip()
        
        if question.lower() in ['q', 'quit']:
            print("🤖 Travel Agent: Safe travels! I'll remember our conversation for next time.")
            break
        
        if not question:
            continue
        
        print("🤖 Travel Agent: ", end='')
        response = agent(question)
        print("\n")


## Step 4: First Session - Learning User Preferences

In [51]:
# Conversation where user shares travel preferences
start_travel_session(session_id)

🌍 Travel Agent Ready - Session: travel_session1
Type 'q' or 'quit' to end the conversation



👤 You:  Hi, I'm Sarah and I'm planning a family vacation for next summer


🤖 Travel Agent: Hi Sarah! Welcome! I'm excited to help you plan your family vacation for next summer. 🌴✈️

To give you the best recommendations, I'd love to learn more about your trip. Could you tell me:

1. **How many people** will be traveling? (Adults and children's ages?)
2. **Where are you departing from?**
3. **Do you have any destinations in mind**, or are you open to suggestions?
4. **What's your approximate budget** for the trip?
5. **What kind of activities** does your family enjoy? (Beach relaxation, adventure, theme parks, cultural experiences, nature, etc.)
6. **Any dietary restrictions, mobility concerns, or other special needs** I should keep in mind?
7. **How long** are you thinking for the trip?

Feel free to share as much or as little as you'd like, and we can start building your perfect family getaway!



👤 You:  We're a family of 4 - me, my husband, and two kids aged 8 and 12. Our budget is around $8000 for the whole trip. We love beaches and the kids are really into water sports.


🤖 Travel Agent: How wonderful, Sarah! A family of four with beach lovers and water sports enthusiasts – that gives us some fantastic options to work with! Let me save these details so I can always provide you with personalized recommendations.
Tool #1: opensearch_memory


...

╭───────────────────── Add long-term memory for session travel_session1, agent travel_agent ──────────────────────╮
│ Sarah's family has chosen Hawaii for their summer vacation. Trip duration: 7 days. Kids have never been         │
│ snorkeling before - this will be their first experience. IMPORTANT: 8-year-old child has a severe seafood       │
│ allergy - must be careful about restaurant selections.                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Memory Stored ─────────────────────────────────────────────────╮
│ ✅ Memory stored successfully:                                                                                  │
│ 🔑 Memory ID: Xyvc5poB8SWL6T84sNEh                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

I've made a special note about your 8-year-old's seafood allergy – that's so important, especially in Hawaii where seafood is everywhere. Here's how we can plan around that:

---

## 🌺 Your 7-Day Hawaii Family Adventure

### Island Recommendation: **Maui**
Perfect for families with beginners – calmer waters, amazing snorkel spots, and lots of non-seafood dining options.

### 🤿 Snorkeling for First-Timers
- **Kapalua Bay** – Calm, shallow, perfect for kids learning to snorkel
- **Turtle Town** – Gentle waters and sea turtles! 🐢
- Consider a **family snorkel tour** with guides who provide instruction and gear sized for kids

### 🍽️ Navigating the Seafood Allergy
Great news – Hawaii has excellent allergy awareness! Here's my advice:
- **Resorts with kitchenettes** – gives you control over some meals
- **Restaurants I recommend**: 
  - Many Italian, Mexican, and farm-to-table spots in Maui handle allergies well
  - Chains like **Monkeypod Kitchen** and **Kihei Caffe** are known for accommo

👤 You:  q


🤖 Travel Agent: Safe travels! I'll remember our conversation for next time.


## Step 5: New Session - Demonstrating Memory Recall

In [52]:
# Agent will previous conversations from previous conversation
# Note: The session_id will be different
start_travel_session('travel_session2')

🌍 Travel Agent Ready - Session: travel_session2
Type 'q' or 'quit' to end the conversation



👤 You:  Hi, it's Sarah again. I wanted to follow up on our Hawaii trip planning.


🤖 Travel Agent: Welcome back, Sarah! Let me quickly check my notes from our previous conversations about your Hawaii trip planning.
Tool #1: opensearch_memory


╭──────────────────────────────────────────────── Search Results ─────────────────────────────────────────────────╮
│                              Long term memories                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓                                   │
│ ┃ Memory ID            ┃ Content                                            ┃                                   │
│ ┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩                                   │
│ │ YCvc5poB8SWL6T841dF2 │ Sarah's 8-year-old child has a severe seafood      │                                   │
│ │                      │ allergy requiring careful restaurant selection     │                                   │
│ │                      │ during ...                                         │                                   │
│ │ Wivb5poB8SWL6T84s9FZ │ Sarah's family consists of 4 members: Sarah, her   │                                   │
│ │                      │ husband, and two children aged 8 and 12.           │                                   │
│ │ WSvb5poB8SWL6T84s9FZ │ Client Sarah is planning a 7-day family vacation   │                                   │
│ │                      │ to Hawaii for summer with a budget of              │                                   │
│ │                      │ approximately...                                   │                                   │
│ │ Wyvb5poB8SWL6T84s9FZ │ Sarah's family loves beaches and the children are  │                                   │
│ │                      │ interested in water sports. The kids have never    │                                   │
│ │                      │ be...                                              │                                   │
│ └──────────────────────┴────────────────────────────────────────────────────┘                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Wonderful, Sarah! I have all our previous notes right here. Let me recap what we've discussed so far about your Hawaii vacation:

**Your Trip Details:**
- 📅 **Duration:** 7-day family vacation to Hawaii this summer
- 💰 **Budget:** Approximately $8,000
- 👨‍👩‍👧‍👦 **Family:** 4 members - you, your husband, and two children (ages 8 and 12)

**Preferences & Important Notes:**
- 🏖️ Your family loves beaches
- 🏄 The kids are interested in water sports
- 🤿 This will be the children's **first snorkeling experience** - how exciting!
- ⚠️ **Critical:** Your 8-year-old has a **severe seafood allergy**, so we need to be very careful with restaurant selections

How can I help you today? Are you ready to start booking, or did you have questions about specific resorts, activities, or dining options that can safely accommodate your child's allergy?



👤 You:  That's amazing you remembered everything! Do you have any resort recommendations?


🤖 Travel Agent: I'm so glad those notes are helpful! Based on your family's profile, here are my top resort recommendations for your Hawaii trip:

---

## 🌺 Top Family Resort Recommendations

### 1. **Aulani, A Disney Resort & Spa** (Oahu - Ko Olina)
- **Why it's perfect for you:** World-class kids' programs for both your 8 and 12-year-old, beautiful lagoon beach perfect for beginner snorkeling, and Disney is renowned for handling food allergies with extreme care
- **Budget fit:** Mid-to-higher range, but excellent value for families
- **Highlights:** Private lagoon, water slides, character experiences, dedicated allergy-trained chefs

### 2. **Grand Hyatt Kauai Resort & Spa** (Kauai)
- **Why it's perfect for you:** Stunning beach access, saltwater lagoon ideal for first-time snorkelers, excellent kids' camp
- **Budget fit:** Works well within your $8,000 budget
- **Highlights:** Multiple pools, nearby Poipu Beach for water sports, restaurants with strong allergen protocols

### 3. **F

👤 You:  q


🤖 Travel Agent: Safe travels! I'll remember our conversation for next time.
